In [1]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


prices = pd.read_excel('/Users/smiths4/Documents/WhiteHat Training/Objectives work/London Rental Prices/Prices v2.xls', sheet_name='Rent')

new_header = prices.iloc[2]
prices = prices[3:]
prices.columns = new_header

prices.rename(columns={'Borough':'Borough'
                      ,'Sum of Count of rents':'SampleSize'
                      ,'Average Rent':'MeanRent'}
                      ,inplace=True)

prices.columns = prices.columns.astype(str)
prices['Borough'] = prices['Borough'].str.strip()
prices = prices.drop(['Sum of Total Rent'],axis=1)
# prices

In [2]:
pop = pd.read_excel('/Users/smiths4/Documents/WhiteHat Training/Objectives work/London Rental Prices/Population.xlsx', sheet_name = 'Borough', header=1)

pop.rename(columns={'Area name': 'Borough'
                   ,'Population per square kilometre':'Pop/km2'
                   ,'GLA Population estimate/ projection':'Pop'}, inplace=True)
pop['Borough'] = pop['Borough'].str.strip()

pop = pop[['Borough','Pop/km2','Pop']]
pop = pop[1:34]
# pop

In [3]:
income = pd.read_excel('/Users/smiths4/Documents/WhiteHat Training/Objectives work/London Rental Prices/Income.xls', sheet_name = 'All', header=4)
pd.set_option('display.max_rows', None)
income = income[221:255]

income.columns = income.columns.astype(str)

income.rename(columns={'Description': 'Borough'
                   ,'Mean': 'MeanSalary'
                   ,'10': '10thPcSalary'
                   ,'25': '25thPcSalary'
                   ,'75': '75thPcSalary'
                   ,'90': '90thPcSalary'
                   ,'Median': 'MedianSalary'}, inplace=True)
income = income[['Borough'
                 ,'MeanSalary'
                ]]
income['Borough'] = income['Borough'].str.strip()
# income

In [4]:
dwellings = pd.read_excel('/Users/smiths4/Documents/WhiteHat Training/Objectives work/London Rental Prices/Dwellings.xlsx')
dwellings.columns = dwellings.columns.astype(str)

dwellings = dwellings[['Area name','2019']]
dwellings.rename(columns={'Area name': 'Borough'
                   ,'2019': 'Dwellings/ha'}, inplace=True)
# dwellings

In [5]:
gcses = pd.read_excel('/Users/smiths4/Documents/WhiteHat Training/Objectives work/London Rental Prices/GCSEs.xlsx',sheet_name='2018-19')
gcses

new_header = gcses.iloc[0]
gcses = gcses[1:]
gcses.columns = new_header

gcses.columns.values[1] = 'Borough'
gcses.columns.values[3] = 'GCSEscore'
gcses = gcses[2:35]
gcses = gcses[['Borough','GCSEscore']]
# gcses

In [6]:
crime = pd.read_excel('/Users/smiths4/Documents/WhiteHat Training/Objectives work/London Rental Prices/Crimes.xlsx',sheet_name='Crime Rates')

new_header = crime.iloc[0]
crime = crime[1:]
crime.columns = new_header

crime = crime[1:35]

crime = crime[['Borough','2016-17']]

import numpy as np
crime = crime.iloc[:,np.r_[0,1]]

crime.rename(columns={'2016-17': 'Crimes/1000'}, inplace=True)
# crime

In [7]:
growth = pd.read_excel('/Users/smiths4/Documents/WhiteHat Training/Objectives work/London Rental Prices/Houseprices.xls',sheet_name='Mean')
# growth
growth = growth[['Area','Year ending Dec 2012','Year ending Dec 2016','Year ending Dec 2017']]
growth.rename(columns={'Area':'Borough'
                     ,'Year ending Dec 2012': 'HousePrice2012'
                     ,'Year ending Dec 2016': 'HousePrice2016'
                     ,'Year ending Dec 2017': 'HousePrice2017'}, inplace=True)
growth = growth[1:34]

growth['5YearGrowth'] = growth.apply(lambda x: x['HousePrice2017']/x['HousePrice2012']-1,axis=1)
growth['1YearGrowth'] = growth.apply(lambda x: x['HousePrice2017']/x['HousePrice2016']-1,axis=1)

growth = growth.iloc[:,np.r_[0,3,4,5]]
# growth

In [8]:
tenure = pd.read_excel('/Users/smiths4/Documents/WhiteHat Training/Objectives work/London Rental Prices/Tenure.xlsx', sheet_name='2018')

tenure.loc[0,'Unnamed: 1'] = 'Borough'

new_header = tenure.iloc[0]
tenure = tenure[3:35]
tenure.columns = new_header

import numpy as np
tenure = tenure.iloc[:,np.r_[1,4,5]]

tenure.rename(columns={'Rented from Local Authority or Housing Association':'Social'
                      ,'Rented from Private landlord':'Private'}
                      ,inplace=True)


tenure['Private'] = tenure['Private'].replace('-','')
tenure['Private'] = tenure['Private'].astype(int)

tenure['Social'] = tenure['Social'].replace('-','')
tenure['Social'] = tenure['Social'].astype(int)

tenure['Total'] = tenure.apply(lambda x: x['Social']+x['Private']-1,axis=1)
tenure['SocialHousing'] = tenure.apply(lambda x: x['Social']/x['Total'],axis=1)

tenure = tenure[['Borough','SocialHousing']]
# tenure

In [9]:
df = pd.merge(prices
             ,income
             ,on='Borough'
             ,how='inner')

df = pd.merge(df
             ,pop
             ,on='Borough'
             ,how='inner')

df = pd.merge(df
             ,dwellings
             ,on='Borough'
             ,how='inner')

df = pd.merge(df
             ,gcses
             ,on='Borough'
             ,how='inner')

df = pd.merge(df
             ,crime
             ,on='Borough'
             ,how='inner')

df = pd.merge(df
             ,growth
             ,on='Borough'
             ,how='inner')

df = pd.merge(df
             ,tenure
             ,on='Borough'
             ,how='inner')

pd.set_option('display.max_columns', None)
df2 = df

df2[['SampleSize','MeanRent']] = df2[['SampleSize','MeanRent']].replace('-','')
df2[['SampleSize','MeanRent']] = df2[['SampleSize','MeanRent']].replace('..','')
df2['MeanRent'] = df2['MeanRent'].astype(int)
df2 = df2.replace('', np.nan)
df2.dropna(inplace=True)
df2

,Borough,SampleSize,MeanRent,MeanSalary,Pop/km2,Pop,Dwellings/ha,GCSEscore,Crimes/1000,HousePrice2017,5YearGrowth,1YearGrowth,SocialHousing
0,Barking and Dagenham,920,1204,26692,5892.712927,212773,20.060938,46.4,70.899932,3.015181e+05,0.661518,0.042679,0.594022
1,Barnet,1530,1503,38728,4577.022673,397049,17.630965,57.1,55.814423,6.675931e+05,0.482655,0.142743,0.355130
2,Bexley,890,1089,36261,4126.712502,249999,15.305227,49.6,45.607942,3.577789e+05,0.586846,0.050358,0.407708
3,Brent,1370,1498,33765,7791.775459,336859,27.860433,50.2,70.789693,5.787048e+05,0.464607,0.118277,0.444868
4,Bromley,2260,1309,43824,2216.226991,332733,9.303900,50.8,52.748792,5.026228e+05,0.509286,0.051201,0.410844
5,Camden,680,2218,55173,11594.546772,252637,48.463248,48.6,103.135498,1.099876e+06,0.408741,0.034449,0.555878
6,Croydon,2190,1137,34666,4523.463089,391296,18.618871,45.5,67.425400,3.996452e+05,0.521956,0.074159,0.476818
7,Ealing,1870,1570,37550,6315.390930,350784,24.359833,50.9,69.010128,5.781099e+05,0.453724,0.044144,0.304831
8,Enfield,610,1291,31967,4177.765256,337697,15.359280,46.5,58.543430,4.638059e+05,0.571687,0.030007,0.420502
9,Greenwich,1670,1478,41411,6048.988138,286322,22.680354,45.3,71.570527,4.628196e+05,0.576660,0.039036,0.568452
